In [4]:
import numpy as np
from sklearn.linear_model import LinearRegression

def create_x(filename,List_01):
    npd=np.genfromtxt(filename,delimiter=',')
    npd = np.delete(npd, 0, 0).T #刪除第0列再轉置
    npd_x_final=np.array(npd[0]).T#挑某幾列當我的x 還要再轉回來
    #List_01=create01List(filename)
    for index in range(0,len(List_01)):
        if List_01[index]==1:
            x_add = np.array(npd[index])
            npd_x_final = np.c_[npd_x_final,x_add.T]
    npd_x_final = np.delete(npd_x_final.T, 0, 0).T #把一開始隨便加的刪掉再轉回來
    #print(npd_x_final)
    return(npd_x_final)

def create_y(filename):
    npd=np.genfromtxt(filename,delimiter=',')
    npd = np.delete(npd, 0, 0).T #刪除第0列再轉置
    npd_y_final=np.array(npd[len(npd.T[1])-1]).T#挑某幾列當我的x 還要再轉回來
    return(npd_y_final)

import random
def create01List(filename):
    list_random = [ random.random() for i in range(len(np.genfromtxt(filename,delimiter=',')[1])-1)]
    for index in range(0,len(list_random)):
        if list_random[index]<=0.5:
            list_random[index]=0
        else:
            list_random[index]=1
    return(list_random)

#交配 FUNCTION
def cross(cross_rate,chromosome_1,chromosome_2):
    random_rate=random.random()
#     print(random_rate)
    if(random_rate>cross_rate):
#         print("cross")
        point_1=round(random.random()*len(chromosome_1))
        point_2=round(random.random()*len(chromosome_2))

        while(point_1>=point_2):
            point_1=round(random.random()*len(chromosome_1))
            point_2=round(random.random()*len(chromosome_2))

#         print("cross_point1: "+str(point_1))
#         print("cross_point2: "+str(point_2))
#         print("cross_len_chromosome_1= "+str(len(chromosome_1)))
#         print("cross_len_chromosome_2= "+str(len(chromosome_2)))

        tmp=0
        for index in range(point_1,point_2):
            tmp=chromosome_1[index]
            chromosome_1[index]=chromosome_2[index]
            chromosome_2[index]=tmp
    return chromosome_1
#         print(chromosome_1)  
#         print(chromosome_2)
        
#mutation fucnition
def mutation(mutation_rate,chromosome):
    
    random_rate=random.random()
    if(random_rate>mutation_rate):
#         print("mutation_start")
        point_1=round(random.random()*(len(chromosome))-1)
        point_2=round(random.random()*(len(chromosome))-1)
        while(point_1==point_2):
            point_1=round(random.random()*(len(chromosome))-1)
            point_2=round(random.random()*(len(chromosome))-1)
#         print("mu_point1= "+str(point_1))
#         print("mu_point2= "+str(point_2))
#         print("mu_chromsome_len= "+str(len(chromosome)))

        if(chromosome[point_1]==0):
            chromosome[point_1]=1
        else:
            chromosome[point_1]=0

        if(chromosome[point_2]==0):
            chromosome[point_2]=1
        else:
            chromosome[point_2]=0
    return chromosome
    

In [5]:
#init create population chromosome(100)--function input：filename & population, output：population_list 
def create_population(filename,population):
    population_list=[]
    for index in range(0,population):
        population_list.append(create01List(filename))
    return population_list

#input population(100) 種染色體， output：算出每個y 並且抓出前rank_number名 --function
def countY_rank(filename,rank_number,population_list):
    #rank_number=4
    #population=100
    result=[]#記錄所有population與產生的y
    best_result=[]#記錄最好的population&y

    for index in population_list:
        npd_x=create_x(filename,index)
        npd_y=create_y(filename)

        lm=LinearRegression()
        lm.fit(npd_x,npd_y)

        #模型績效(MSE & R^2)
        mse=np.mean((lm.predict(npd_x)-npd_y)**2)
        R_squared=lm.score(npd_x,npd_y)
        adj_r_squared = R_squared - (1 - R_squared) * (npd_x.shape[1] / (npd_x.shape[0] - npd_x.shape[1] - 1))
        result.append([[adj_r_squared],[index]])

    result_final=[]

    for index in range(0,rank_number):
        result_final.append(sorted(result,reverse=True)[index])
    best_result.append(result_final[0])

    x_final=[]#記錄前十五名的染色體
    for index in result_final:
        x_final.append(index[1][0])
    return [x_final,best_result]

#產生新的X(上一代最好的前15個交配+突變產生新的15個+隨機產生85個新的x)--function
def create_new_x(filename,rank_number,population,cross_rate,mutation_rate,x_final):
    new_x=[]
    #先把前15名的染色體做交配&突變->產生新15個染色體
    for index in range(0,rank_number):
        other_ch=round(random.random()*(rank_number-1))
        while(index==other_ch):
            other_ch=round(random.random()*(rank_number-1)) #變矩陣要減1
        tmp=cross(cross_rate,x_final[index],x_final[other_ch])
        new_x.append(mutation(mutation_rate,tmp))

    #隨機產生第16~100的染色體
    for index in range(0,population-rank_number):
        new_x.append(create01List(filename))
    return new_x



In [14]:
#GA + regression ->main
#init parameter
filename='0050_test.csv'
iteration=30
iteration_now=0
population=10
rank_number=4

cross_rate=0.1
mutation_rate=0.1

y_target=1.5
y_best=-1
y_now=-1
x_best=[]

population_list=create_population(filename,population)

print("start GA")
while(not(iteration_now>=iteration or y_best>y_target)):
    iteration_now=iteration_now+1
    print("\niteration_now= "+str(iteration_now))
    Y=countY_rank(filename,rank_number,population_list)
    y_now=Y[1][0][0][0]
    print("y_now= "+str(y_now))
    if y_now>y_best:
        y_best=y_now
        x_best=Y[1][0][1][0]
    population_list=create_new_x(filename,rank_number,population,cross_rate,mutation_rate,Y[0])

print("iteration_now= "+str(iteration_now))
print("y_best= "+str(y_best))
print("x_best= "+str(x_best))
    
      
    

start GA

iteration_now= 1
y_now= 0.996177805079

iteration_now= 2
y_now= 0.996173127124

iteration_now= 3
y_now= 0.996177828729

iteration_now= 4
y_now= 0.99617528621

iteration_now= 5
y_now= 0.996173421769

iteration_now= 6
y_now= 0.996169853304

iteration_now= 7
y_now= 0.996176394698

iteration_now= 8
y_now= 0.996173466221

iteration_now= 9
y_now= 0.996172624222

iteration_now= 10
y_now= 0.996175735484

iteration_now= 11
y_now= 0.996174437428

iteration_now= 12
y_now= 0.996177895764

iteration_now= 13
y_now= 0.996171821789

iteration_now= 14
y_now= 0.996174047467

iteration_now= 15
y_now= 0.996174812901

iteration_now= 16
y_now= 0.996178024295

iteration_now= 17
y_now= 0.996171383887

iteration_now= 18
y_now= 0.996175290599

iteration_now= 19
y_now= 0.996174327085

iteration_now= 20
y_now= 0.996165735329

iteration_now= 21
y_now= 0.996179594234

iteration_now= 22
y_now= 0.996178658807

iteration_now= 23
y_now= 0.996169293158

iteration_now= 24
y_now= 0.996174397286

iteration_now= 2

In [13]:
#將最好的x_best再跑一次model
npd_x=create_x(filename,x_best)
npd_y=create_y(filename)

lm=LinearRegression()
lm.fit(npd_x,npd_y)

#模型績效(MSE & R^2)
mse=np.mean((lm.predict(npd_x)-npd_y)**2)
R_squared=lm.score(npd_x,npd_y)
adj_r_squared = R_squared - (1 - R_squared) * (npd_x.shape[1] / (npd_x.shape[0] - npd_x.shape[1] - 1))
print(adj_r_squared)





0.996159808488


In [ ]:
import seaborn as sns  
import matplotlib.pyplot as plt   
# visualize the relationship between the features and the response using scatterplots  
sns.pairplot(data, x_vars=['TV','Radio','Newspaper'], y_vars='Sales', size=7, aspect=0.8)  
plt.show()#注意必须加上这一句，否则无法显示。  